<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebookd/03_patch_tst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Setup

The following setup is provided as a basic example for initializing the notebook environment. It includes necessary imports, optional configuration, and a placeholder for data loading or downloading.

This section is **not part of the core model logic**, and the code here may vary depending on your environment or data access method.

## Setup Environment


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

Cloning into 'Walmart-Recruiting-Store-Sales-Forecasting'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 367 (delta 88), reused 52 (delta 22), pack-reused 209 (from 1)
Receiving objects: 100% (367/367), 6.92 MiB | 4.03 MiB/s, done.
Resolving deltas: 100% (185/185), done.
/content/Walmart-Recruiting-Store-Sales-Forecasting


In [3]:
%%capture
!pip install -r requirements.txt

In [4]:
%%capture
from google.colab import userdata
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

In [5]:
from google.colab import userdata
wandb_api_key = userdata.get('WANDB_API_LOGIN')
!wandb login {wandb_api_key}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## Load and Split Data

In [36]:
from src import data_loader, processing
import importlib
importlib.reload(processing)

dataframes = data_loader.load_raw_data()
df = processing.run_preprocessing(dataframes, process_test=False, merge_features=False, merge_stores=False)['train']
X_train, y_train, X_valid, y_valid = processing.split_data_by_ratio(df, separate_target=True)

print(f"Shapes of train_df and valid_df: {train_df.shape}, {valid_df.shape}")

Data loading complete.
Shapes of train_df and valid_df: (337256, 3), (84314, 3)


In [7]:
train_df

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,2,2010-02-05,50605.27,False
2,1,3,2010-02-05,13740.12,False
3,1,4,2010-02-05,39954.04,False
4,1,5,2010-02-05,32229.38,False
...,...,...,...,...,...
337251,22,27,2012-04-13,1995.50,False
337252,22,28,2012-04-13,1125.95,False
337253,22,29,2012-04-13,6768.77,False
337254,22,30,2012-04-13,4457.84,False


In [8]:
valid_df

,Store,Dept,Date,Weekly_Sales,IsHoliday
337256,22,32,2012-04-13,6236.25,False
337257,22,33,2012-04-13,6179.73,False
337258,22,34,2012-04-13,24139.31,False
337259,22,35,2012-04-13,4432.28,False
337260,22,36,2012-04-13,2256.69,False
...,...,...,...,...,...
421565,45,93,2012-10-26,2487.80,False
421566,45,94,2012-10-26,5203.31,False
421567,45,95,2012-10-26,56017.47,False
421568,45,97,2012-10-26,6817.48,False


# Train

In [57]:
import pandas as pd
from neuralforecast import NeuralForecast
from sklearn.base import BaseEstimator, RegressorMixin

from models.neural_forecast_models import NeuralForecastModels


In [55]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
import torch
from neuralforecast.losses.pytorch import MAE
from src.utils import wmae as compute_wmae


model = [PatchTST(
  input_size=52,
  h=132,
  max_steps= 25 * 104,
  batch_size=32,
  random_seed=42,
  enable_progress_bar=False
)]


mdl = NeuralForecastModels(model, ['PatchTST'], one_model=True)

mdl.fit(X_train, y_train)



INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type              | Params | Mode 
-----------------------------------------------------------
0 | loss         | MAE               | 0      | train
1 | padder_train | ConstantPad1d     | 0      | train
2 | scaler       | TemporalNorm      | 0      | train
3 | model        | PatchTST_backbone | 501 K  | train
-----------------------------------------------------------
501 K     Trainable params
3         Non-trainable params
501 K     Total params
2.008     Total estimated model params size (MB)
90        Modules in train mode
0         Modules in

In [56]:
print(compute_wmae(mdl.predict(X_valid), y_valid, X_valid['IsHoliday']))

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1561.3032175903659


In [40]:
X_valid

,Store,Dept,Date,IsHoliday
337256,22,32,2012-04-13,False
337257,22,33,2012-04-13,False
337258,22,34,2012-04-13,False
337259,22,35,2012-04-13,False
337260,22,36,2012-04-13,False
...,...,...,...,...
421565,45,93,2012-10-26,False
421566,45,94,2012-10-26,False
421567,45,95,2012-10-26,False
421568,45,97,2012-10-26,False


In [28]:
merged = valid_df.merge(
    forecast[['unique_id', 'ds', 'PatchTST']],
    on=['unique_id', 'ds'],
    how='left'
)

print(merged.head())


          ds         y unique_id        NBEATS
0 2012-04-13   6236.25     22_32   5225.511230
1 2012-04-13   6179.73     22_33   7481.007812
2 2012-04-13  24139.31     22_34  23797.615234
3 2012-04-13   4432.28     22_35   5344.318359
4 2012-04-13   2256.69     22_36   2444.783203


In [17]:
nan_rows_specific = merged[merged[['y', 'PatchTST']].isna().any(axis=1)]
print(nan_rows_specific)


              ds      y unique_id  PatchTST
10842 2012-05-11  15.00      7_99       NaN
12778 2012-05-11 -30.00     36_36       NaN
13534 2012-05-18  19.33      3_83       NaN
27362 2012-06-15  15.00     33_49       NaN
33254 2012-06-29  10.00     33_49       NaN
35088 2012-07-06  59.76     16_99       NaN
39182 2012-07-13   0.02     33_99       NaN
39361 2012-07-13   0.02     36_99       NaN
40929 2012-07-20  29.88     15_99       NaN
44423 2012-07-27  29.88     23_99       NaN
45073 2012-07-27   0.02     33_99       NaN
48024 2012-08-03   0.02     33_99       NaN
50194 2012-08-10  29.88     21_99       NaN
54112 2012-08-17 -29.00     36_71       NaN
56670 2012-08-24  29.88     29_99       NaN
58900 2012-08-31   4.47     19_39       NaN
61862 2012-09-07  13.41     19_39       NaN
62832 2012-09-07   0.04     33_99       NaN
62996 2012-09-07   5.00     36_49       NaN
64371 2012-09-14  29.88     12_99       NaN
64826 2012-09-14  20.86     19_39       NaN
65280 2012-09-14  -1.27     25_9

In [29]:
merged.fillna(0, inplace=True)

In [30]:
from src.utils import wmae as compute_wmae

print(compute_wmae(merged['PatchTST'], merged['y'], is_holiday))
# print(len(merged), len(is_holiday))

1578.3519622417095
